# Scraping of VGR video gaming forum
We start by importing the request library and getting the main page.

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
url = 'https://www.vgr.com/forum/forum/37-gaming-forum/'

cookies = {
    'ips4_hasJS': 'true',
    'ips4_ipsTimezone': 'Europe/Amsterdam',
    '__qca': 'P0-1185121426-1683547568961',
    '_ga': 'GA1.2.770526334.1683547569',
    '_ga_FG1X7MCBQC': 'GS1.1.1683547568.1.0.1683547568.0.0.0',
    '_gat_gtag_UA_117339006_2': '1',
    '_gid': 'GA1.2.1917616821.1683547569',
    'properSessionStorage': 'eyJ1dWlkIjoiNTY5NTdhNmYtOWY4Yi00ODA0LThjNTItY2ZhM2IwNjg2NWIwIiwiZGVwdGgiOjEsInJlZmVycmVyIjoiaHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8iLCJnY2xpZCI6IiIsImZiY2xpZCI6IiIsInV0bV9jYW1wYWlnbiI6IiIsInV0bV9zb3VyY2UiOiIiLCJ1dG1fbWVkaXVtIjoiIiwidXRtX3Rlcm0iOiIiLCJ1dG1fY29udGVudCI6IiIsInV0bV90ZW1wbGF0ZSI6IiIsInV0bV9yZWZlcnJlciI6IiIsInV0bV9hZHNldCI6IiIsInV0bV9zdWJpZCI6IiIsInJldmVudWUiOjAsImJpZF9hdmciOnt9LCJub19iaWRfY250Ijp7fSwiYXVjdGlvbl9jb3VudCI6MCwibGFzdF90aHJlc2hvbGQiOjB9',
    'q82nja': '1',
    'ips4_IPSSessionFront': '5ba2282c61e6e7eecba2a17b90321637',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15',
}

r = requests.get(url=url, headers=headers, cookies=cookies)
r

<Response [200]>

In [4]:
soup = BeautifulSoup(r.content)

# The forum consists of multiple threads. Every thread is in a div item with class=ipsDataItem_main, so we will find all of the threads first.
threads = soup.find_all("div", {"class": "ipsDataItem_main"})
len(threads)

threads[0].find('a')['href']

'https://www.vgr.com/forum/topic/60-last-game-played/'

In [5]:
# Scrape and get links
def get_post_link(url: str) -> list[str]:
    """
    Scrapes the page and returns the link of the post
    :param url: the url of the main page with all threads
    :return: All urls to scrape
    """
    all_links_on_page: list[str] = []

    # Scrape
    r = requests.get(url, headers=headers, cookies=cookies)

    # Next, we use BeautifulSoup to parse the HTML and look into the contents.
    soup = BeautifulSoup(r.content, 'html.parser')

    # The forum consists of multiple threads. Every thread is in a div item with class=ipsDataItem_main, so we will find all of the threads first.
    threads = soup.find_all("div", {"class": "ipsDataItem_main"})

    # Get links from threads
    for thread in threads:
        # Get the link to the full thread
        if thread.find('a'):
            link_to_thread = thread.find('a')['href']
            all_links_on_page.append(link_to_thread)

        # If it cannot be found, go to the next thread
        else:
            continue

    return all_links_on_page

In [6]:
from time import sleep
from random import randint

# List of all links to scrape:
urls_to_scrape: list[list[str]] = []

# Main URL
url = 'https://www.vgr.com/forum/forum/37-gaming-forum/'

urls_to_scrape.append(get_post_link(url))

for page in range(2, 170):  # There are 170 pages
    sleep(randint(1,7))
    url = f'https://www.vgr.com/forum/forum/37-gaming-forum/page/{page}/'
    urls_to_scrape.append(get_post_link(url))

Next download the forums themselves.

## Scrape all threads
This is a new web page, so we create a new request.

In [21]:
from pandas import DataFrame
from typing import Optional
import pandas as pd

video_games_data: Optional[DataFrame] = None

for list_of_threads in urls_to_scrape:
    for url in list_of_threads:
        sleep(randint(1,7))
        r = requests.get(url, cookies=cookies, headers=headers)

        soup_thread = BeautifulSoup(r.content, 'html.parser')

        title_list = []
        poster_id_list = []
        post_text_list = []

        # Get the title
        title = soup_thread.find('span', {'class': 'ipsType_break ipsContained'})
        if title is not None:
            title = title.text

        # Find all replies
        replies = soup_thread.find_all('article')

        for reply in replies:
            # Get the poster ID
            poster_id = reply.find('a', {'class': 'ipsType_break'})
            if poster_id is not None:
                poster_id = poster_id.text

            # Get the text
            text = reply.find('div', {'data-role': 'commentContent'})
            if text is None:
                continue

            poster_id_list.append(poster_id)
            post_text_list.append(text.text)

        data = {
            'thread_title':title,
            'poster_id': poster_id_list,
            'text': post_text_list,
        }

        if video_games_data is None:
            video_games_data = pd.DataFrame(data)
        else:
            video_games_data = pd.concat([video_games_data, pd.DataFrame(data)])

## Data cleaning
First, we delete \n from the text.

In [22]:
video_games_data['text']

0    \n\nWhat was the last game you played and for ...
1    \n\nIf we're talking real gaming then Ni No Ku...
2    \n\nLast game I played on ps3 was minecraft. S...
3    \n\nThe last game I played was Counter Strike:...
4    \n\nThis is hilarious because I'm in the exact...
                           ...                        
6    \n\nThe trouble is--and I know I'm playing dev...
0    \n\nFor me, it's gotta be Der Reise as it was ...
0    \n\nOne thing that annoys me about DLC is the ...
1    \n\nI don't buy DLC very often unless it is fo...
2    \n\nGreat topic! Hmm, I think The Shivering Is...
Name: text, Length: 38469, dtype: object

In [44]:
video_games_data['text'] = video_games_data['text'].apply(lambda x: x.replace("\n", " "))
video_games_data['thread_title'] = video_games_data['thread_title'].apply(lambda x: x.replace("\n", " "))

In [48]:
video_games_data.reset_index(drop=True, inplace=True)

In [49]:
video_games_data.to_csv("forum_vgr.csv")